# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to use [requests](http://www.python-requests.org/en/master/) to download HTML pages from a website?
* How to select content on a webpage with [lxml](http://lxml.de/)? 

You can either use Spark DataFrame or [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) to do the assignment. In comparison, pandas.DataFrame has richer APIs, but is not good at distributed computing.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of HTML, DOM, and XPath. I found that this is a good resource: [https://data-lessons.github.io/library-webscraping/](https://data-lessons.github.io/library-webscraping/). Please take a look at

* [Selecting content on a web page with XPath
](https://data-lessons.github.io/library-webscraping/xpath/)
* [Web scraping using Python: requests and lxml](https://data-lessons.github.io/library-webscraping/04-lxml/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. One day, you want to analyze CS faculty data and answer two interesting questions:

1. Who are the CS faculty members?
2. What are their research interests?

To do so, the first thing is to figure out what data to collect.

## Task 1: SFU CS Faculty Members

You find that there is a web page in the CS school website, which lists all the faculty members as well as their basic information. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawling Web Page

A web page is essentially a file stored in a remote machine (called web server). You can use [requests](http://www.python-requests.org/en/master/) to open such a file and read data from it. Please complete the following code to download the HTML page and save it as a text file (like [this](./faculty.txt)). 

In [2]:
import requests

# 1. Download the webpage
response = requests.get('https://www.sfu.ca/computing/people/faculty.html')

# 2. Save it as a text file (named faculty.txt)
file = open('faculty.txt', 'w')
file.write(response.text)
file.close()


### (b) Extracting Structured Data

An HTML page follows the Document Object Model (DOM). It models an HTML page as a tree structure wherein each node is an object representing a part of the page. The nodes can be searched and extracted programmatically using XPath. Please complete the following code to transform the above HTML page to a CSV file (like [this](./faculty_table.csv)). 

In [3]:
import lxml.html 

# 1. Open faculty.txt
def readfromfile(filename):
	with open(filename, 'r') as content_file:
		filecontent = content_file.read()
		return(filecontent)

filecontent = readfromfile('faculty.txt')

# 2. Parse the HTML page as a tree structure
doc = lxml.html.fromstring(filecontent)
nodepath = "//div[@class='textimage section']//div[@class='text']"
SFU_HOMEPAGE = 'http://www.sfu.ca'

# 3. Extract related content from the tree using XPath
sfu_prof_records = []

prof_els = doc.xpath(nodepath)
for prof_el in prof_els:
		name = ''
		designation = ''
		profile = ''
		homepage = ''
		research_interests = []

		for children in prof_el.getiterator():
			if(children.tag  == 'h4'):
				prof = (children.text_content().split('\n'))[0].split(',')
				prof_name = prof[0]
				name = prof_name.title()
				prof_designation = prof[1]
				prof_designation = prof_designation.strip()
				designation = prof_designation.title()
				if name == 'Associate Director': continue
# 				print('Name: ' + name)
# 				print('Designation:' + prof_designation.title())

			elif (children.tag == 'p'):
				for profile_details in children.getiterator():
					link = profile_details.get('href')
					title = profile_details.text_content()
					if title == 'Profile & Contact Information' and link is not None:
						if not link.startswith(SFU_HOMEPAGE): link = SFU_HOMEPAGE + link
						# print('Profile:' + link)
						profile = link
					elif (title == 'Home Page'):
						homepage = link
						# print('HomePage:' + homepage)
		
		row_tuple = (name, designation, profile, homepage)
		sfu_prof_records.append(row_tuple)

# 4. Save the extracted content as an csv file (named faculty_table.csv)
import csv

with open('faculty_table.csv','w') as out:
	csv_out=csv.writer(out)
	csv_out.writerow(['name', 'designation', 'profile', 'homepage'])
	for row in sfu_prof_records:
		csv_out.writerow(row)

## Task 2: Research Interests

Suppose you want to know the research interests of each faculty. However, the above crawled web page does not contain such information. 

### (a) Crawling Web Page

You notice that such information can be found on the profile page of each faculty. For example, you can find the research interests of Dr. Jiannan Wang from [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please complete the following code to download the profile pages and save them as text files. There are 56 faculties, so you need to download 56 web pages in total. 

In [4]:
import requests

# 1. Download the profile pages of 56 faculties
for prof in sfu_prof_records:
    (name, designation, profile, homepage) = prof
#     print(profile)

# 2. Save each page as a text file
    response = requests.get(profile)
    file = open(name + '.txt', 'w')
    file.write(response.text)
    file.close()
    

### (b) Extracting Structured Data

Please complete the following code to extract the research interests of each faculty, and generate a file like [this](./faculty_more_table.csv). 

In [6]:
import lxml.html 

# 1. Open each text file and parse it as a tree structure 
def readfromfile(filename):
	with open(filename, 'r') as content_file:
		filecontent = content_file.read()
		return(filecontent)

def getinterests(filename):
    content = readfromfile(filename)
    doc = lxml.html.fromstring(content)

    li_sections = doc.xpath("//div[@class='text parbase section']//div[@class='ruled']")
    if not li_sections:
        li_sections = doc.xpath("//div[@class='text parbase section']//div[@class='listed']")

    for section in li_sections:
        tables = section.getchildren()
        if tables:
            h2 = tables[0]
            if h2.text_content().strip().lower() == 'research interests':
                ul = tables[1]
                interests = ul.text_content().split('\n')
                interests = list(filter(None, interests))
                return(interests)

# 2. Extract the research interests from each tree using XPath
faculty_more_tuples = []

# 3. Add the extracted content to faculty_table.csv    
# 4. Generate a new CSV file, named faculty_more_table.csv
import csv
with open('faculty_more_table.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['name', 'designation', 'profile', 'homepage', 'research_interests'])
    for prof in sfu_prof_records:
        (name, designation, profile, homepage) = prof
        filename = (name + '.txt')
        interests = getinterests(filename)
        row_tuple = (name, designation, profile, homepage, interests)
        csv_out.writerow(row_tuple)



## Complete python code

I have also included a .py file that I written initially that can be run as a standalone.

In [2]:
'''
import lxml.html, requests
import csv

def savetoTXTfile(filename, content):
	file = open(filename, 'w')
	file.write(content)
	file.close()

def savetoCSVfile(filename, content):
	with open(filename,'w') as out:
		csv_out=csv.writer(out)
		csv_out.writerow(['name', 'designation', 'profile', 'homepage', 'research_interests'])
		for row in content:
			csv_out.writerow(row)

def readfromfile(filename):
	with open(filename, 'r') as content_file:
		filecontent = content_file.read()
		return(filecontent)

def getinterests(profile_url):
	response = requests.get(profile_url)
	content = response.text
	doc = lxml.html.fromstring(content)

	li_sections = doc.xpath("//div[@class='text parbase section']//div[@class='ruled']")
	if not li_sections:
		li_sections = doc.xpath("//div[@class='text parbase section']//div[@class='listed']")

	for section in li_sections:
		tables = section.getchildren()
		if tables:
			h2 = tables[0]
			if h2.text_content().strip().lower() == 'research interests':
				ul = tables[1]
				interests = ul.text_content().split('\n')
				interests = list(filter(None, interests))
				return(interests)


def makelocalcopyofpage(url, filename):
	response = requests.get(url)
	content = response.text
	savetoTXTfile(filename, content)

def crawlusingfile(filename):
	filecontent = readfromfile(filename)
	doc = lxml.html.fromstring(filecontent)
	nodepath = "//div[@class='textimage section']//div[@class='text']"
	SFU_HOMEPAGE = 'http://www.sfu.ca'
	prof_els = doc.xpath(nodepath)
	sfu_prof_records = []

	for prof_el in prof_els:
		name = ''
		designation = ''
		profile = ''
		homepage = ''
		research_interests = []

		for children in prof_el.getiterator():
			if(children.tag  == 'h4'):
				prof = (children.text_content().split('\n'))[0].split(',')
				prof_name = prof[0]
				name = prof_name.title()
				prof_designation = prof[1]
				prof_designation = prof_designation.strip()
				designation = prof_designation.title()
				if name == 'Associate Director': continue
				# print('Name: ' + name)
				# print('Designation:' + prof_designation.title())

			elif (children.tag == 'p'):
				for profile_details in children.getiterator():
					link = profile_details.get('href')
					title = profile_details.text_content()
					if title == 'Profile & Contact Information' and link is not None:
						if not link.startswith(SFU_HOMEPAGE): link = SFU_HOMEPAGE + link
						# print('Profile:' + link)
						profile = link
						research_interests = getinterests(profile)
						if not research_interests: research_interests = []
						# print("Interests: {}".format(research_interests))
					elif (title == 'Home Page'):
						homepage = link
						# print('HomePage:' + homepage)
		
		row_tuple = (name, designation, profile, homepage, research_interests)
		sfu_prof_records.append(row_tuple)

	savetoCSVfile('faculty_more_table.csv', sfu_prof_records)

def main():
	filename = 'faculty.txt'
	faculty_page = 'https://www.sfu.ca/computing/people/faculty.html'
	makelocalcopyofpage(faculty_page, filename)
	crawlusingfile(filename)

if __name__ == '__main__':
  main()
'''

## Submission

Complete the code in this [notebook](A1.ipynb), and submit it to the CourSys activity [Assignment 1](https://courses.cs.sfu.ca/2018sp-cmpt-733-g1/+a1/).